# 要約 
このJupyter Notebookは、LMSYSのChatbot Arenaコンペティションのために、チャットボットの応答に対する人間の好みを予測するモデルを開発することを目的としています。具体的には、LLM（大規模言語モデル）を使用し、「winner_model_a」「winner_model_b」「winner_tie」といった分類を予測するために、ロジスティック回帰的な手法を用いています。

### 主な問題
- ユーザーが2つのモデルによる応答の中からどちらを好むかを予測する問題に取り組んでいます。データからは、各応答がどれだけ好まれるかを評価するためにログ損失（log loss）をメトリクスとして使用しています。

### 使用手法とライブラリ
- **Hugging Face Transformers**: モデルのトレーニングや推論に使用され、特に`LlamaModel`や`AutoTokenizer`が利用されています。
- **Pandas、NumPy**: データ処理に使用されており、データの読み込みや変換、数値計算を行っています。
- **PEFT (Parameter-Efficient Fine-Tuning)**: パラメータの効率的な調整を行うために、Low-Rank Adaptation（LoRA）を使用しています。これにより、モデルの性能を保ちながら、必要なメモリを削減しています。
- **Scikit-learn**: 評価指標としてのログロスと精度を計算するためにインポートされています。

### 処理の流れ
1. 必要なライブラリのインストールとインポート。
2. 訓練データの読み込みと前処理。各応答のラベル設定を行います。
3. トークナイザーの設定を行い、データセットをトークン化。
4. モデルの準備（`Llama3ForSFT`クラス）、LoRA設定を行い、事前学習されたモデルを読み込みます。
5. トレーニングの引数を設定し、トレーニングを実行。

最終的に、提案されたアプローチによって、与えられたプロンプトに対して最適な応答を予測する能力を持つモデルが訓練されています。評価結果として、ログロス値が0.9231、リーダーボードでのスコアが0.936となっています。

---


# 用語概説 
以下は、機械学習・深層学習の初心者がつまずきそうな専門用語の解説です。特に、ノートブック特有のドメイン知識や、実務を経験していないと馴染みのないものに焦点を当てています。

### 専門用語の解説

1. **ログロス (Log Loss)**: 
   確率的な予測と実際のラベルとの間の差異を測定する評価指標です。特に二項分類や多項分類の問題で使われ、低い値が高い予測性能を示します。予測確率が正解ラベルに近づくほど、ログロスは小さくなります。

2. **アテンションマスク (Attention Mask)**:
   トランスフォーマーモデルにおいて、モデルがどの部分に注意を向けるべきかを示すものです。通常、入力がパディングされた場合、そのパディング部分に注意を向けないように1と0の値を持ちます（1は注意する部分、0は無視する部分）。

3. **トークナイザー (Tokenizer)**:
   テキストデータを機械が理解できる形式に変換するツールです。単語やサブワードのトークンに分割し、それに対するインデックス（ID）を付与します。事前訓練済みのトークナイザーは、大規模な言語モデルに特化したものが多いです。

4. **データコレータ (Data Collator)**:
   各バッチに対するデータの前処理を行うクラスです。例えば、各サンプルの長さを揃えるためのパディングを行うことなどが含まれます。特にバッチサイズが異なるサンプルが混在している場合には重要です。

5. **勾配蓄積 (Gradient Accumulation)**:
   メモリ制約で大きなバッチサイズを使えない場合、小さなバッチサイズの勾配を複数回計算し、それらを累積してから重みの更新を行う手法です。これにより、実質的に大きなバッチサイズで訓練した時と同等の効果が得られます。

6. **PEFT (Parameter-Efficient Fine-Tuning)**:
   訓練済みのモデルパラメータを効率的に微調整する手法のことです。特に、大規模モデルのパラメータ数が多い場合に、一部のパラメータのみを更新することで、計算資源を節約しながら高精度なモデルを得ることを目的としています。

7. **ロジット (Logits)**:
   ニューラルネットワークの出力で、クラスに対する生の信号やスコアを指します。ロジットは通常、ソフトマックス関数を通じて確率に変換され、確率分布を形成します。

8. **因果言語モデル (Causal Language Model)**:
   次の単語を予測するために、前の単語の情報のみを使用するモデルのことです。文の左側から右側に向けて条件付けが行われるため、過去の単語に基づいて次の単語を生成します。

9. **デバイス (Device)**:
   モデルやデータが置かれるコンピュータハードウェアのこと。一般にGPU（グラフィックス処理ユニット）を使用することが多く、計算速度を飛躍的に向上させます。`torch.device('cuda')`のようにして指定します。

10. **事前学習済みモデル (Pre-trained Model)**:
    大量のデータで訓練されたモデルのこと。特定のタスクに対して再訓練（ファインチューニング）することで、少ないデータで効果的なモデルを構築するために利用されます。

これらの用語は、特にノートブックの機械学習や深層学習の流れにおいて、理解を深めるために重要です。

---


<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Result
- [Inference Code](https://www.kaggle.com/code/shelterw/sft-llama-3-8b-inference)    

- [Base Model: llama-3-8b-Instruct-bnb-4bit](https://huggingface.co/unsloth/llama-3-8b-Instruct-bnb-4bit)

| subset | log loss |
| - | - |
| Eval | 0.9231|
| LB | 0.936 |

## Note
If you want to reproduce the code, please note the following:
- use all data
- set per_device_train_batch_size=4
- 1 epoch using A10 took ~15h


</div>
<div class="column-right">

# 日本語訳

## 結果
- [推論コード](https://www.kaggle.com/code/shelterw/sft-llama-3-8b-inference)    

- [ベースモデル: llama-3-8b-Instruct-bnb-4bit](https://huggingface.co/unsloth/llama-3-8b-Instruct-bnb-4bit)

| サブセット | ログロス |
| - | - |
| 評価 | 0.9231|
| LB | 0.936 |

## 注意事項
コードを再現したい場合は、以下の点に注意してください:
- すべてのデータを使用
- `per_device_train_batch_size=4`を設定
- A10を使用して1エポックで約15時間かかります



</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!pip install git+https://github.com/huggingface/transformers
!pip install -U bitsandbytes accelerate peft
```

</div>
<div class="column-right">

# 日本語訳

```python
!pip install git+https://github.com/huggingface/transformers  # Hugging FaceのTransformersライブラリをインストールします。
!pip install -U bitsandbytes accelerate peft  # bitsandbytes、accelerate、peftを最新バージョンにアップグレードしてインストールします。
```

</div>
</details>

In [ ]:
!pip install git+https://github.com/huggingface/transformers  # Hugging FaceのTransformersライブラリをインストールします。
!pip install -U bitsandbytes accelerate peft  # bitsandbytes、accelerate、peftを最新バージョンにアップグレードしてインストールします。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import os
import copy
from dataclasses import dataclass

import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
from datasets import Dataset
from scipy.special import softmax
from sklearn.preprocessing import LabelEncoder
from transformers import (
    BitsAndBytesConfig,
    LlamaModel,
    AutoTokenizer,
    PreTrainedTokenizerBase, 
    EvalPrediction,
    Trainer,
    TrainingArguments,
    DataCollatorForSeq2Seq,
    AutoModel
)
from transformers.modeling_outputs import CausalLMOutputWithPast
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from sklearn.metrics import log_loss, accuracy_score
```

</div>
<div class="column-right">

# 日本語訳

```python
import os  # オペレーティングシステムに関連する機能をインポートします。
import copy  # オブジェクトのコピーをサポートする機能をインポートします。
from dataclasses import dataclass  # データクラスを作成するためのデコレーターをインポートします。

import torch  # PyTorchの基本ライブラリをインポートします。
import torch.nn as nn  # PyTorchのニューラルネットワークモジュールをインポートします。
import torch.nn.functional as F  # ニューラルネットワーク用の関数をインポートします。
import pandas as pd  # データ処理用のPandasライブラリをインポートします。
import numpy as np  # 数値計算用のNumPyライブラリをインポートします。
from datasets import Dataset  # データセットを扱うためのクラスをインポートします。
from scipy.special import softmax  # softmax関数をインポートします。
from sklearn.preprocessing import LabelEncoder  # ラベルエンコーディングを行うためのクラスをインポートします。
from transformers import (  # Transformersライブラリから必要なクラスをインポートします。
    BitsAndBytesConfig,
    LlamaModel,
    AutoTokenizer,
    PreTrainedTokenizerBase, 
    EvalPrediction,
    Trainer,
    TrainingArguments,
    DataCollatorForSeq2Seq,
    AutoModel
)
from transformers.modeling_outputs import CausalLMOutputWithPast  # 過去の出力を含む因果言語モデルの出力をインポートします。
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType  # PEFTに関するクラスや関数をインポートします。
from sklearn.metrics import log_loss, accuracy_score  # 評価指標としてのログロスと精度をインポートします。
```

</div>
</details>

In [ ]:
import os  # オペレーティングシステムに関連する機能をインポートします。
import copy  # オブジェクトのコピーをサポートする機能をインポートします。
from dataclasses import dataclass  # データクラスを作成するためのデコレーターをインポートします。

import torch  # PyTorchの基本ライブラリをインポートします。
import torch.nn as nn  # PyTorchのニューラルネットワークモジュールをインポートします。
import torch.nn.functional as F  # ニューラルネットワーク用の関数をインポートします。
import pandas as pd  # データ処理用のPandasライブラリをインポートします。
import numpy as np  # 数値計算用のNumPyライブラリをインポートします。
from datasets import Dataset  # データセットを扱うためのクラスをインポートします。
from scipy.special import softmax  # softmax関数をインポートします。
from sklearn.preprocessing import LabelEncoder  # ラベルエンコーディングを行うためのクラスをインポートします。
from transformers import (  # Transformersライブラリから必要なクラスをインポートします。
    BitsAndBytesConfig,
    LlamaModel,
    AutoTokenizer,
    PreTrainedTokenizerBase, 
    EvalPrediction,
    Trainer,
    TrainingArguments,
    DataCollatorForSeq2Seq,
    AutoModel
)
from transformers.modeling_outputs import CausalLMOutputWithPast  # 過去の出力を含む因果言語モデルの出力をインポートします。
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType  # PEFTに関するクラスや関数をインポートします。
from sklearn.metrics import log_loss, accuracy_score  # 評価指標としてのログロスと精度をインポートします。

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

### Configurations

</div>
<div class="column-right">

# 日本語訳

### 設定


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
TRAIN_CSV = "/kaggle/input/lmsys-chatbot-arena/train.csv"
model_path = "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
MAX_LENGTH = 1024
target_columns = ['winner_model_a', 'winner_model_b', 'winner_tie']
columns_to_vectorize = ["prompt", "response_a", "response_b"]

train = pd.read_csv(TRAIN_CSV)
train = train.head(100)
train['label'] = train[target_columns].idxmax(axis=1) 
label_encoder = LabelEncoder()
train['label'] = label_encoder.fit_transform(train['label'])
train = train[columns_to_vectorize + ['label']]
```

</div>
<div class="column-right">

# 日本語訳

```python
TRAIN_CSV = "/kaggle/input/lmsys-chatbot-arena/train.csv"  # 訓練データのCSVファイルのパスを指定します。
model_path = "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit"  # 使用するモデルのパスを指定します。
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # GPUが利用可能かどうかを確認してデバイスを設定します。
MAX_LENGTH = 1024  # 最大入力長を設定します。
target_columns = ['winner_model_a', 'winner_model_b', 'winner_tie']  # 予測対象の列を指定します。
columns_to_vectorize = ["prompt", "response_a", "response_b"]  # ベクトル化する列を指定します。

train = pd.read_csv(TRAIN_CSV)  # CSVファイルから訓練データを読み込みます。
train = train.head(100)  # 最初の100件を取得します。
train['label'] = train[target_columns].idxmax(axis=1)  # 各行のラベルを決定します。
label_encoder = LabelEncoder()  # ラベルエンコーダを初期化します。
train['label'] = label_encoder.fit_transform(train['label'])  # ラベルをエンコードします。
train = train[columns_to_vectorize + ['label']]  # 必要な列だけを保持します。
```

</div>
</details>

In [ ]:
TRAIN_CSV = "/kaggle/input/lmsys-chatbot-arena/train.csv"  # 訓練データのCSVファイルのパスを指定します。
model_path = "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit"  # 使用するモデルのパスを指定します。
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # GPUが利用可能かどうかを確認してデバイスを設定します。
MAX_LENGTH = 1024  # 最大入力長を設定します。
target_columns = ['winner_model_a', 'winner_model_b', 'winner_tie']  # 予測対象の列を指定します。
columns_to_vectorize = ["prompt", "response_a", "response_b"]  # ベクトル化する列を指定します。

train = pd.read_csv(TRAIN_CSV)  # CSVファイルから訓練データを読み込みます。
train = train.head(100)  # 最初の100件を取得します。
train['label'] = train[target_columns].idxmax(axis=1)  # 各行のラベルを決定します。
label_encoder = LabelEncoder()  # ラベルエンコーダを初期化します。
train['label'] = label_encoder.fit_transform(train['label'])  # ラベルをエンコードします。
train = train[columns_to_vectorize + ['label']]  # 必要な列だけを保持します。

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

### Tokenizer and prepare dataset, metrics

</div>
<div class="column-right">

# 日本語訳

### トークナイザーとデータセットの準備、メトリクス


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
tokenizer = AutoTokenizer.from_pretrained(model_path, force_download = True )

```

</div>
<div class="column-right">

# 日本語訳

```python
tokenizer = AutoTokenizer.from_pretrained(model_path, force_download=True)  # 事前学習済みのトークナイザーをロードします。
```

</div>
</details>

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_path, force_download=True)  # 事前学習済みのトークナイザーをロードします。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
tokenizer.add_eos_token = True
tokenizer.padding_side = 'right'

LABEL_IDS = [tokenizer(i, add_special_tokens=False)["input_ids"][0] for i in ['a', 'b', 'tie']]

def tokenize(example, tokenizer):
    prompt = tokenizer('<prompt>: ' + " ".join(eval(example['prompt'], {"null": ""})), add_special_tokens=False)["input_ids"]
    response_a = tokenizer('\n\n<response_a>: ' + " ".join(eval(example['response_a'], {"null": ""})), add_special_tokens=False)["input_ids"]
    response_b = tokenizer('\n\n<response_b>: ' + " ".join(eval(example['response_b'], {"null": ""})), add_special_tokens=False)["input_ids"]
    if len(prompt+response_a+response_b) > MAX_LENGTH:
        prompt = tokenizer('<prompt>: ' + eval(example['prompt'], {"null": ""})[-1], add_special_tokens=False)["input_ids"][:256]
        response_a = tokenizer('\n\n<response_a>: ' + eval(example['response_a'], {"null": ""})[-1], add_special_tokens=False)["input_ids"][:512]
        response_b = tokenizer('\n\n<response_b>: ' + eval(example['response_b'], {"null": ""})[-1], add_special_tokens=False)["input_ids"][:512]
    extra_prompt = tokenizer('\n\n---------\nWhich is the better response for the prompt ? a or b or tie ?\n\nAnswer: ', add_special_tokens=False)["input_ids"]

    label_token_id = LABEL_IDS[int(example['label'])]
    input_ids = [tokenizer.bos_token_id] + prompt + response_a + response_b + extra_prompt + [label_token_id] + [tokenizer.eos_token_id]
    attention_mask = len(input_ids)*[1]
    labels = [-100]* len([tokenizer.bos_token_id] + prompt + response_a + response_b + extra_prompt) + [label_token_id] + [tokenizer.eos_token_id]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

```

</div>
<div class="column-right">

# 日本語訳

```python
tokenizer.add_eos_token = True  # 終了トークンを追加します。
tokenizer.padding_side = 'right'  # パディングを右側に設定します。

# ラベルIDsをトークナイザーを使って作成します。
LABEL_IDS = [tokenizer(i, add_special_tokens=False)["input_ids"][0] for i in ['a', 'b', 'tie']]

def tokenize(example, tokenizer):
    # プロンプト、応答a、および応答bをトークン化します。
    prompt = tokenizer('<prompt>: ' + " ".join(eval(example['prompt'], {"null": ""})), add_special_tokens=False)["input_ids"]
    response_a = tokenizer('\n\n<response_a>: ' + " ".join(eval(example['response_a'], {"null": ""})), add_special_tokens=False)["input_ids"]
    response_b = tokenizer('\n\n<response_b>: ' + " ".join(eval(example['response_b'], {"null": ""})), add_special_tokens=False)["input_ids"]
    # 入力が最大長を超える場合は入力を切り詰めます。
    if len(prompt+response_a+response_b) > MAX_LENGTH:
        prompt = tokenizer('<prompt>: ' + eval(example['prompt'], {"null": ""})[-1], add_special_tokens=False)["input_ids"][:256]
        response_a = tokenizer('\n\n<response_a>: ' + eval(example['response_a'], {"null": ""})[-1], add_special_tokens=False)["input_ids"][:512]
        response_b = tokenizer('\n\n<response_b>: ' + eval(example['response_b'], {"null": ""})[-1], add_special_tokens=False)["input_ids"][:512]
    extra_prompt = tokenizer('\n\n---------\nWhich is the better response for the prompt ? a or b or tie ?\n\nAnswer: ', add_special_tokens=False)["input_ids"]

    # ラベルに対応するトークンIDを取得します。
    label_token_id = LABEL_IDS[int(example['label'])]
    input_ids = [tokenizer.bos_token_id] + prompt + response_a + response_b + extra_prompt + [label_token_id] + [tokenizer.eos_token_id]
    attention_mask = len(input_ids) * [1]  # アテンションマスクを作成します。
    labels = [-100] * len([tokenizer.bos_token_id] + prompt + response_a + response_b + extra_prompt) + [label_token_id] + [tokenizer.eos_token_id]
    # トークナイズされたデータを返します。
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }
```

</div>
</details>

In [ ]:
tokenizer.add_eos_token = True  # 終了トークンを追加します。
tokenizer.padding_side = 'right'  # パディングを右側に設定します。

# ラベルIDsをトークナイザーを使って作成します。
LABEL_IDS = [tokenizer(i, add_special_tokens=False)["input_ids"][0] for i in ['a', 'b', 'tie']]

def tokenize(example, tokenizer):
    # プロンプト、応答a、および応答bをトークン化します。
    prompt = tokenizer('<prompt>: ' + " ".join(eval(example['prompt'], {"null": ""})), add_special_tokens=False)["input_ids"]
    response_a = tokenizer('\n\n<response_a>: ' + " ".join(eval(example['response_a'], {"null": ""})), add_special_tokens=False)["input_ids"]
    response_b = tokenizer('\n\n<response_b>: ' + " ".join(eval(example['response_b'], {"null": ""})), add_special_tokens=False)["input_ids"]
    # 入力が最大長を超える場合は入力を切り詰めます。
    if len(prompt+response_a+response_b) > MAX_LENGTH:
        prompt = tokenizer('<prompt>: ' + eval(example['prompt'], {"null": ""})[-1], add_special_tokens=False)["input_ids"][:256]
        response_a = tokenizer('\n\n<response_a>: ' + eval(example['response_a'], {"null": ""})[-1], add_special_tokens=False)["input_ids"][:512]
        response_b = tokenizer('\n\n<response_b>: ' + eval(example['response_b'], {"null": ""})[-1], add_special_tokens=False)["input_ids"][:512]
    extra_prompt = tokenizer('\n\n---------\nWhich is the better response for the prompt ? a or b or tie ?\n\nAnswer: ', add_special_tokens=False)["input_ids"]

    # ラベルに対応するトークンIDを取得します。
    label_token_id = LABEL_IDS[int(example['label'])]
    input_ids = [tokenizer.bos_token_id] + prompt + response_a + response_b + extra_prompt + [label_token_id] + [tokenizer.eos_token_id]
    attention_mask = len(input_ids) * [1]  # アテンションマスクを作成します。
    labels = [-100] * len([tokenizer.bos_token_id] + prompt + response_a + response_b + extra_prompt) + [label_token_id] + [tokenizer.eos_token_id]
    # トークナイズされたデータを返します。
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def load_data(df, tokenizer):
    raw_datasets = Dataset.from_pandas(df)
    tokenized_datasets = raw_datasets.map(
        tokenize, 
        remove_columns=raw_datasets.column_names,
        fn_kwargs={'tokenizer': tokenizer}
    )
    return tokenized_datasets

def compute_metrics(pred):
    logits, labels = pred
    preds = logits.argmax(axis=-1)
    label_tokens_ids = np.array(LABEL_IDS)
    index_mapping = {value.item(): idx for idx, value in enumerate(label_tokens_ids)}
    labels = labels[np.isin(labels, label_tokens_ids)]
    labels = np.array([index_mapping[label.item()] for label in labels])
    acc = accuracy_score(labels, preds)
    probs = softmax(logits, axis=-1)
    log_loss_ = log_loss(labels, probs)
    return {'accuracy': acc, 'log_loss': log_loss_}

n_splits = 5
fold_idx = 0
ds = load_data(train, tokenizer)
folds = [
    (
        [i for i in range(len(ds)) if i % n_splits != fold_idx],
        [i for i in range(len(ds)) if i % n_splits == fold_idx]
    ) 
    for fold_idx in range(n_splits)
]
train_idx, eval_idx = folds[fold_idx]
```

</div>
<div class="column-right">

# 日本語訳

```python
def load_data(df, tokenizer):
    # DataFrameをデータセットに変換します。
    raw_datasets = Dataset.from_pandas(df)
    tokenized_datasets = raw_datasets.map(
        tokenize, 
        remove_columns=raw_datasets.column_names,
        fn_kwargs={'tokenizer': tokenizer}  # トークナイザーを引数として渡します。
    )
    return tokenized_datasets

def compute_metrics(pred):
    logits, labels = pred  # 予測とラベルを取得します。
    preds = logits.argmax(axis=-1)  # 最大のロジット値を持つインデックスを取得します。
    label_tokens_ids = np.array(LABEL_IDS)  # ラベルIDsを配列に変換します。
    index_mapping = {value.item(): idx for idx, value in enumerate(label_tokens_ids)}  # インデックスマッピングを作成します。
    labels = labels[np.isin(labels, label_tokens_ids)]  # ラベルがラベルIDのいずれかに含まれている場合にフィルタリングします。
    labels = np.array([index_mapping[label.item()] for label in labels])  # マッピングを適用してラベルを変換します。
    acc = accuracy_score(labels, preds)  # 精度を計算します。
    probs = softmax(logits, axis=-1)  # ソフトマックスを適用して確率を計算します。
    log_loss_ = log_loss(labels, probs)  # ログロスを計算します。
    return {'accuracy': acc, 'log_loss': log_loss_}  # 精度とログロスを返します。

n_splits = 5  # データセットを分割するためのスプリット数を設定します。
fold_idx = 0  # フォールドインデックスを初期化します。
ds = load_data(train, tokenizer)  # データセットをロードします。
folds = [
    (
        [i for i in range(len(ds)) if i % n_splits != fold_idx],  # 訓練データのインデックス
        [i for i in range(len(ds)) if i % n_splits == fold_idx]  # 評価データのインデックス
    ) 
    for fold_idx in range(n_splits)  # 各フォールドのインデックスに対して反復処理します。
]
train_idx, eval_idx = folds[fold_idx]  # 訓練インデックスと評価インデックスを取得します。
```

</div>
</details>

In [ ]:
def load_data(df, tokenizer):
    # DataFrameをデータセットに変換します。
    raw_datasets = Dataset.from_pandas(df)
    tokenized_datasets = raw_datasets.map(
        tokenize, 
        remove_columns=raw_datasets.column_names,
        fn_kwargs={'tokenizer': tokenizer}  # トークナイザーを引数として渡します。
    )
    return tokenized_datasets

def compute_metrics(pred):
    logits, labels = pred  # 予測とラベルを取得します。
    preds = logits.argmax(axis=-1)  # 最大のロジット値を持つインデックスを取得します。
    label_tokens_ids = np.array(LABEL_IDS)  # ラベルIDsを配列に変換します。
    index_mapping = {value.item(): idx for idx, value in enumerate(label_tokens_ids)}  # インデックスマッピングを作成します。
    labels = labels[np.isin(labels, label_tokens_ids)]  # ラベルがラベルIDのいずれかに含まれている場合にフィルタリングします。
    labels = np.array([index_mapping[label.item()] for label in labels])  # マッピングを適用してラベルを変換します。
    acc = accuracy_score(labels, preds)  # 精度を計算します。
    probs = softmax(logits, axis=-1)  # ソフトマックスを適用して確率を計算します。
    log_loss_ = log_loss(labels, probs)  # ログロスを計算します。
    return {'accuracy': acc, 'log_loss': log_loss_}  # 精度とログロスを返します。

n_splits = 5  # データセットを分割するためのスプリット数を設定します。
fold_idx = 0  # フォールドインデックスを初期化します。
ds = load_data(train, tokenizer)  # データセットをロードします。
folds = [
    (
        [i for i in range(len(ds)) if i % n_splits != fold_idx],  # 訓練データのインデックス
        [i for i in range(len(ds)) if i % n_splits == fold_idx]  # 評価データのインデックス
    ) 
    for fold_idx in range(n_splits)  # 各フォールドのインデックスに対して反復処理します。
]
train_idx, eval_idx = folds[fold_idx]  # 訓練インデックスと評価インデックスを取得します。

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

### Model

</div>
<div class="column-right">

# 日本語訳

### モデル


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
from transformers import AutoModel, MistralPreTrainedModel, MistralModel
class Llama3ForSFT(MistralPreTrainedModel):
    _tied_weights_keys = ["lm_head.weight"]
    def __init__(self, config):
        super().__init__(config)
        self.model = MistralModel(config)
        self.vocab_size = config.vocab_size
        self.lm_head = nn.Linear(config.hidden_size, config.vocab_size, bias=False)
        self.post_init()

    def forward(
        self,
        input_ids= None,
        attention_mask= None,
        position_ids = None,
        past_key_values= None,
        inputs_embeds= None,
        labels= None,
        use_cache= None,
        output_attentions= None,
        output_hidden_states = None,
        return_dict= None,
        cache_position = None,
    ):
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            position_ids=position_ids,
            past_key_values=past_key_values,
            inputs_embeds=inputs_embeds,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
            cache_position=cache_position,
        )
        hidden_states = outputs[0]
#         if self.config.pretraining_tp > 1:
#             lm_head_slices = self.lm_head.weight.split(self.vocab_size // self.config.pretraining_tp, dim=0)
#             logits = [F.linear(hidden_states, lm_head_slices[i]) for i in range(self.config.pretraining_tp)]
#             logits = torch.cat(logits, dim=-1)
#         else:
        logits = self.lm_head(hidden_states)
        logits = logits.float()

        loss = None
        if labels is not None:
            # Shift so that tokens < n predict n
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            # Flatten the tokens
            loss_fct = nn.CrossEntropyLoss()
            shift_logits = shift_logits.view(-1, self.config.vocab_size)
            shift_labels = shift_labels.view(-1)
            # Enable model parallelism
            shift_labels = shift_labels.to(shift_logits.device)

            label_tokens_ids = torch.tensor(LABEL_IDS,device=shift_labels.device)
            index_mapping = {value.item(): idx for idx, value in enumerate(label_tokens_ids)}
            true_labels = shift_labels[torch.isin(shift_labels, label_tokens_ids)]
            true_labels = torch.tensor([index_mapping[label.item()] for label in true_labels], device=true_labels.device)
            true_logits = shift_logits[torch.isin(shift_labels, label_tokens_ids)][:,label_tokens_ids]
            loss = loss_fct(true_logits, true_labels)

        return CausalLMOutputWithPast(
            loss=loss,
            logits=true_logits,
        )
```

</div>
<div class="column-right">

# 日本語訳

```python
from transformers import AutoModel, MistralPreTrainedModel, MistralModel  # モデル関連のクラスをインポートします。
class Llama3ForSFT(MistralPreTrainedModel):  # MistralPreTrainedModelを拡張したLlama3ForSFTクラスを定義します。
    _tied_weights_keys = ["lm_head.weight"]  # 重みの結合キーを定義します。

    def __init__(self, config):
        super().__init__(config)  # 親クラスの初期化を行います。
        self.model = MistralModel(config)  # Mistralモデルを初期化します。
        self.vocab_size = config.vocab_size  # 語彙サイズを取得します。
        self.lm_head = nn.Linear(config.hidden_size, config.vocab_size, bias=False)  # ロジット層を初期化します。
        self.post_init()  # モデルの初期化を完了します。

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        position_ids=None,
        past_key_values=None,
        inputs_embeds=None,
        labels=None,
        use_cache=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
        cache_position=None,
    ):
        outputs = self.model(
            input_ids=input_ids,  # 入力IDを渡します。
            attention_mask=attention_mask,
            position_ids=position_ids,
            past_key_values=past_key_values,
            inputs_embeds=inputs_embeds,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
            cache_position=cache_position,
        )
        hidden_states = outputs[0]  # 隠れ状態を取得します。
        logits = self.lm_head(hidden_states)  # 隠れ状態からロジットを計算します。
        logits = logits.float()  # ロジットを浮動小数点型に変換します。

        loss = None  # 初期化
        if labels is not None:
            # ラベルがある場合の処理を行います。
            # シフトしてトークンがn未満を予測します。
            shift_logits = logits[..., :-1, :].contiguous()  # ロジットをシフトします。
            shift_labels = labels[..., 1:].contiguous()  # ラベルをシフトします。
            # トークンをフラット化します。
            loss_fct = nn.CrossEntropyLoss()  # クロスエントロピー損失関数を定義します。
            shift_logits = shift_logits.view(-1, self.config.vocab_size)  # ロジットを再成形します。
            shift_labels = shift_labels.view(-1)  # ラベルを再成形します。
            # モデル並行処理を有効にします。
            shift_labels = shift_labels.to(shift_logits.device)  # デバイスにラベルを移動します。

            label_tokens_ids = torch.tensor(LABEL_IDS, device=shift_labels.device)  # ラベルIDsをテンソルに変換します。
            index_mapping = {value.item(): idx for idx, value in enumerate(label_tokens_ids)}  # インデックスマッピングを作成します。
            true_labels = shift_labels[torch.isin(shift_labels, label_tokens_ids)]  # 正しいラベルを取得します。
            true_labels = torch.tensor([index_mapping[label.item()] for label in true_labels], device=true_labels.device)  # マッピングを適用します。
            true_logits = shift_logits[torch.isin(shift_labels, label_tokens_ids)][:, label_tokens_ids]  # 正しいロジットを取得します。
            loss = loss_fct(true_logits, true_labels)  # 損失を計算します。

        return CausalLMOutputWithPast(  # 結果を返します。
            loss=loss,
            logits=true_logits,
        )
```

</div>
</details>

In [ ]:
from transformers import AutoModel, MistralPreTrainedModel, MistralModel  # モデル関連のクラスをインポートします。
class Llama3ForSFT(MistralPreTrainedModel):  # MistralPreTrainedModelを拡張したLlama3ForSFTクラスを定義します。
    _tied_weights_keys = ["lm_head.weight"]  # 重みの結合キーを定義します。

    def __init__(self, config):
        super().__init__(config)  # 親クラスの初期化を行います。
        self.model = MistralModel(config)  # Mistralモデルを初期化します。
        self.vocab_size = config.vocab_size  # 語彙サイズを取得します。
        self.lm_head = nn.Linear(config.hidden_size, config.vocab_size, bias=False)  # ロジット層を初期化します。
        self.post_init()  # モデルの初期化を完了します。

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        position_ids=None,
        past_key_values=None,
        inputs_embeds=None,
        labels=None,
        use_cache=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
        cache_position=None,
    ):
        outputs = self.model(
            input_ids=input_ids,  # 入力IDを渡します。
            attention_mask=attention_mask,
            position_ids=position_ids,
            past_key_values=past_key_values,
            inputs_embeds=inputs_embeds,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
            cache_position=cache_position,
        )
        hidden_states = outputs[0]  # 隠れ状態を取得します。
        logits = self.lm_head(hidden_states)  # 隠れ状態からロジットを計算します。
        logits = logits.float()  # ロジットを浮動小数点型に変換します。

        loss = None  # 初期化
        if labels is not None:
            # ラベルがある場合の処理を行います。
            # シフトしてトークンがn未満を予測します。
            shift_logits = logits[..., :-1, :].contiguous()  # ロジットをシフトします。
            shift_labels = labels[..., 1:].contiguous()  # ラベルをシフトします。
            # トークンをフラット化します。
            loss_fct = nn.CrossEntropyLoss()  # クロスエントロピー損失関数を定義します。
            shift_logits = shift_logits.view(-1, self.config.vocab_size)  # ロジットを再成形します。
            shift_labels = shift_labels.view(-1)  # ラベルを再成形します。
            # モデル並行処理を有効にします。
            shift_labels = shift_labels.to(shift_logits.device)  # デバイスにラベルを移動します。

            label_tokens_ids = torch.tensor(LABEL_IDS, device=shift_labels.device)  # ラベルIDsをテンソルに変換します。
            index_mapping = {value.item(): idx for idx, value in enumerate(label_tokens_ids)}  # インデックスマッピングを作成します。
            true_labels = shift_labels[torch.isin(shift_labels, label_tokens_ids)]  # 正しいラベルを取得します。
            true_labels = torch.tensor([index_mapping[label.item()] for label in true_labels], device=true_labels.device)  # マッピングを適用します。
            true_logits = shift_logits[torch.isin(shift_labels, label_tokens_ids)][:, label_tokens_ids]  # 正しいロジットを取得します。
            loss = loss_fct(true_logits, true_labels)  # 損失を計算します。

        return CausalLMOutputWithPast(  # 結果を返します。
            loss=loss,
            logits=true_logits,
        )

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias='none',
    inference_mode=False,
    task_type=TaskType.CAUSAL_LM,
    target_modules=['q_proj', 'k_proj', 'v_proj',], 
)

model = Llama3ForSFT.from_pretrained(
    model_path, 
    torch_dtype=torch.float16, 
)
model.config.use_cache = False
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)
print(model)
model.print_trainable_parameters()
```

</div>
<div class="column-right">

# 日本語訳

```python
peft_config = LoraConfig(  # PEFT設定を作成します。
    r=16,  # ランクを設定します。
    lora_alpha=32,  # alpha値を設定します。
    lora_dropout=0.05,  # ドロップアウト率を設定します。
    bias='none',  # バイアスの設定を行います。
    inference_mode=False,  # 推論モードをオフにします。
    task_type=TaskType.CAUSAL_LM,  # タスクのタイプを因果言語モデルに設定します。
    target_modules=['q_proj', 'k_proj', 'v_proj'],  # 対象となるモジュールを設定します。
)

# 事前学習済みのモデルをロードし、kビット訓練の準備をします。
model = Llama3ForSFT.from_pretrained(
    model_path, 
    torch_dtype=torch.float16,  # モデルのデータ型をfloat16に設定します。
)
model.config.use_cache = False  # キャッシュを無効にします。
model = prepare_model_for_kbit_training(model)  # kビット訓練のための準備をします。
model = get_peft_model(model, peft_config)  # PEFTモデルを取得します。
print(model)  # モデルの情報を表示します。
model.print_trainable_parameters()  # 訓練可能なパラメータを表示します。
```

</div>
</details>

In [ ]:
peft_config = LoraConfig(  # PEFT設定を作成します。
    r=16,  # ランクを設定します。
    lora_alpha=32,  # alpha値を設定します。
    lora_dropout=0.05,  # ドロップアウト率を設定します。
    bias='none',  # バイアスの設定を行います。
    inference_mode=False,  # 推論モードをオフにします。
    task_type=TaskType.CAUSAL_LM,  # タスクのタイプを因果言語モデルに設定します。
    target_modules=['q_proj', 'k_proj', 'v_proj'],  # 対象となるモジュールを設定します。
)

# 事前学習済みのモデルをロードし、kビット訓練の準備をします。
model = Llama3ForSFT.from_pretrained(
    model_path, 
    torch_dtype=torch.float16,  # モデルのデータ型をfloat16に設定します。
)
model.config.use_cache = False  # キャッシュを無効にします。
model = prepare_model_for_kbit_training(model)  # kビット訓練のための準備をします。
model = get_peft_model(model, peft_config)  # PEFTモデルを取得します。
print(model)  # モデルの情報を表示します。
model.print_trainable_parameters()  # 訓練可能なパラメータを表示します。

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

#### Training Arguments

</div>
<div class="column-right">

# 日本語訳

#### トレーニング引数


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
args = TrainingArguments(
    output_dir='output',
    overwrite_output_dir = True,
    evaluation_strategy = "epoch",
    save_strategy = "steps",
    save_steps=200,
    save_total_limit=1,
    logging_strategy="steps",
    logging_steps=10,
    warmup_steps=20,
    optim="adamw_8bit",
    learning_rate=2e-4,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=3,
    num_train_epochs=1,
    fp16=True,
    metric_for_best_model="log_loss",
    greater_is_better = False,
    report_to="none",
)

```

</div>
<div class="column-right">

# 日本語訳

```python
args = TrainingArguments(  # トレーニング引数を設定します。
    output_dir='output',  # 出力ディレクトリを指定します。
    overwrite_output_dir=True,  # 出力ディレクトリを上書きします。
    evaluation_strategy="epoch",  # 評価戦略をエポックごとに設定します。
    save_strategy="steps",  # 保存戦略をステップごとに設定します。
    save_steps=200,  # 200ステップごとに保存します。
    save_total_limit=1,  # 保存するファイル数の制限を設定します。
    logging_strategy="steps",  # ロギング戦略をステップごとに設定します。
    logging_steps=10,  # 10ステップごとにロギングします。
    warmup_steps=20,  # ウォームアップステップを設定します。
    optim="adamw_8bit",  # 最適化手法を設定します。
    learning_rate=2e-4,  # 学習率を設定します。
    per_device_train_batch_size=1,  # デバイスごとの訓練バッチサイズを設定します。
    per_device_eval_batch_size=1,  # デバイスごとの評価バッチサイズを設定します。
    gradient_accumulation_steps=3,  # 勾配蓄積ステップを設定します。
    num_train_epochs=1,  # 訓練エポック数を設定します。
    fp16=True,  # 半精度浮動小数点を使用します。
    metric_for_best_model="log_loss",  # 最良モデルのメトリックをログロスに設定します。
    greater_is_better=False,  # より良いのは小さい方の損失とします。
    report_to="none",  # ロギングの報告先を指定しません。
)
```

</div>
</details>

In [ ]:
args = TrainingArguments(  # トレーニング引数を設定します。
    output_dir='output',  # 出力ディレクトリを指定します。
    overwrite_output_dir=True,  # 出力ディレクトリを上書きします。
    evaluation_strategy="epoch",  # 評価戦略をエポックごとに設定します。
    save_strategy="steps",  # 保存戦略をステップごとに設定します。
    save_steps=200,  # 200ステップごとに保存します。
    save_total_limit=1,  # 保存するファイル数の制限を設定します。
    logging_strategy="steps",  # ロギング戦略をステップごとに設定します。
    logging_steps=10,  # 10ステップごとにロギングします。
    warmup_steps=20,  # ウォームアップステップを設定します。
    optim="adamw_8bit",  # 最適化手法を設定します。
    learning_rate=2e-4,  # 学習率を設定します。
    per_device_train_batch_size=1,  # デバイスごとの訓練バッチサイズを設定します。
    per_device_eval_batch_size=1,  # デバイスごとの評価バッチサイズを設定します。
    gradient_accumulation_steps=3,  # 勾配蓄積ステップを設定します。
    num_train_epochs=1,  # 訓練エポック数を設定します。
    fp16=True,  # 半精度浮動小数点を使用します。
    metric_for_best_model="log_loss",  # 最良モデルのメトリックをログロスに設定します。
    greater_is_better=False,  # より良いのは小さい方の損失とします。
    report_to="none",  # ロギングの報告先を指定しません。
)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

### Training !

</div>
<div class="column-right">

# 日本語訳

### トレーニング !


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import transformers 
transformers.__version__
```

</div>
<div class="column-right">

# 日本語訳

```python
import transformers  # Transformersライブラリをインポートします。
transformers.__version__  # 現在のTransformersのバージョンを表示します。
```

</div>
</details>

In [ ]:
import transformers  # Transformersライブラリをインポートします。
transformers.__version__  # 現在のTransformersのバージョンを表示します。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
trainer = Trainer(
    args=args,
    model=model,
    train_dataset=ds.select(train_idx),
    eval_dataset=ds.select(eval_idx),
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)
trainer.train()
```

</div>
<div class="column-right">

# 日本語訳

```python
trainer = Trainer(  # Trainerインスタンスを作成します。
    args=args,  # トレーニング引数を渡します。
    model=model,  # 使用するモデルを指定します。
    train_dataset=ds.select(train_idx),  # 訓練データセットを指定します。
    eval_dataset=ds.select(eval_idx),  # 評価データセットを指定します。
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),  # データコレータを指定します。
    compute_metrics=compute_metrics,  # メトリクス計算を指定します。
)
trainer.train()  # 訓練を開始します。
```

</div>
</details>

In [ ]:
trainer = Trainer(  # Trainerインスタンスを作成します。
    args=args,  # トレーニング引数を渡します。
    model=model,  # 使用するモデルを指定します。
    train_dataset=ds.select(train_idx),  # 訓練データセットを指定します。
    eval_dataset=ds.select(eval_idx),  # 評価データセットを指定します。
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),  # データコレータを指定します。
    compute_metrics=compute_metrics,  # メトリクス計算を指定します。
)
trainer.train()  # 訓練を開始します。